**Возникшие вопросы**
   1. Почему в задании указано, что необходимо выбрать *датасеты*, а в примерах *бенчмарки*? При использовании бенчмарков теряется весь смысл задания, ведь они уже содержат множество датасетов и метрик с описаниями. 
   
**План проекта**
   1. Выбор датасетов.
   
   2. Определение метрик.
   
   3. Выбор подходов для сравнения.
   
   4. Постановка гипотез.
   
   5. Проведение экспериментов.
   
   6. Выводы.

### Датасеты:
+ Параллельный перевод для удмуртского языка (как языка малой народности) [UDM](https://huggingface.co/datasets/udmurtNLP/flores-250-rus-udm).
+ Парафразы [PAR](https://huggingface.co/datasets/merionum/ru_paraphraser). Кроме того, содержит "непарафразы", что также можно использовать для проверки качества.
+ Всегда актуальная токсичность [TOX](https://huggingface.co/datasets/FredZhang7/toxi-text-3M). Я выбрал именно этот датасет, так как он содержит 50+ языков, и это наибльшая выборка для этой задачи, которую я сумел найти. В ней слишком много английского, турецкого и арабского. 
+ Суммаризация [SUM](https://huggingface.co/datasets/embedding-data/sentence-compression). В этом датасете тексты относительно небольшие и не требуют больших вычислительных мощностей.
+ Мультиклассовая классификация [MGP](https://huggingface.co/datasets/datadrivenscience/movie-genre-prediction). Модель с описанием фильмов и жанрами.
| Датасет | Размер* | Число языков |
|----------|----------|----------|
| flores-250-rus-udm    | 250   | 2   |
| ru_paraphraser    | 7227   | 1   |
| instructor-base    | 37910   | 45   |
|sentence-compression |     2000     |     1       |
|movie-genre-prediction |     54000        |   1      |

### Метрики:
+ *UDM* - косинусная близость русских и удмуртских эмбеддингов. Универсальный энкодер должен хорошо справляться не только с широко представленными языками и успешно извлекать смыслы из всего, что ему передают.
+ *PAR* - считаем косинусную близость для пар с классом "precise paraphrases" и (1 - косинусная близость) для пар с классом "non-paraphrases". Класс "near paraphrases" игнорируем. Метрика, в какой-то степени, дополняет метрику *UDM*, показывая, что модель не просто располагает все эмбеддинги в одной точки векорного пространства.
+ *SUM* - косинусная близость эмбеддингов текста и краткого изложения. Хотим знать, насколько хорошо модель справляется с извлечением смысла из больших текстов.
+ *TOX* - произведение средней точности KNN** и энтропии точностей по классам (по языкам) $A \cdot entropy([a_1, a_2, \ldots, a_n]), a_i = acc_{c_i}$. Метрика показывает, насколько хорошо подель справляется с бинарной классификацией. Если не дополнять вычисления энтропией, то при использовании модели в дальнейшем на каком-то языке модель может показать результат, гораздо ниже ожидаемого. Текущая метрика будет сильно ниже, если модель справляется с предсказаниями на каких-то отдельных языках хуже, чем на других.
+ *MGP* - f1_score macro на предсказания KNN. Проверка на задаче мультиклассовой классификации.

### Подходы:
+ *Word2Vec* - бейзлайн и наиболее примитивный подход. Для получения эмбеддингов предложения, буду брать усреднение по эмбеддингам слов. Я взял веса от предобученной модели [*glove-twitter-25*](https://radimrehurek.com/gensim/models/word2vec.html#other-embeddings).
+ *T5-flan* [google/flan-t5-small](https://huggingface.co/google/flan-t5-small)  Использую усреднение эмбеддингов энкодера, так как в [этой статье](https://arxiv.org/abs/2108.08877) такой подход показал лучший результат для zero-shot. Модель обучалась на достаточно большом корпусе языков.
+ [*InstructOR*](https://arxiv.org/pdf/2212.09741.pdf) - ещё одна T5-based модель, которая обучалась на каком-то невероятно большом корпусе данных и различных задач специально для создания векторных представлений.
+ multilingual-e5-small ##############
+ sentence-transformers/all-MiniLM-L6-v2 ##############
+ Labse ##############

| Модель | Вес | На скольки языках обучена  |
|----------|----------|----------|
| glove-twitter-25              | 110 MB   | 1   |
| google/flan-t5-small          | 308 MB   | 50+   |
| hkunlp/instructor-base        | 439 MB   | ?   |
|intfloat/multilingual-e5-small |     471 MB     |     90+       |
| all-MiniLM-L6-v2              |     91 MB        |   1      |
| setu4993/LaBSE                | **1.88 GB**| **109** |

### Гипотезы:
+ Word2Vec покажет себя хуже всего в среднем по всем метрикам, так как не предназначен для задачи создания векторных представлений текста.
+ Лучше всего себя покажет LaBSE, она содержит наибольшее число параметров и обучена на наибольшем количестве задач.
+ Модель InstructOR покажет себя лучше, чем T5-flan, так как они имеют схожую архитектуру, но она была обучена специально для задачи получения векторных представлений и испытана на широком классе задач. Но, я не смог найти, какое число языков было в обучающей выборке. У google/flan-t5-small это значение достаточно велико.

### Реализация:
В папке *src* находятся следующие файлы:
+ *data_processors.py* - содержит классы для каждого датасета. 
+ *model_processors.py* - для каждого типа модели свой класс-обертка, чтобы можно было использовать общий пайплайн. При этом можно заменять модели на более большие, но в текущих экспериментах я старался брать наименьшие.
+ *metrics.py* - файл с методами, для расчета итоговых метрик.
+ *utils.py* - дополнительные методы.

Более подробное описание классов и методов есть в коде.

### Результаты:

| Модель | UDM | PAR | SUM | TOX | MGP |
|----------|----------|----------|----------|----------|----------|
| glove-twitter-25              |    |    |     |     |     | 
| google/flan-t5-small          |    |    |    |     |     | 
| hkunlp/instructor-base        |    |    |    |     |     | 
|intfloat/multilingual-e5-small |    |    |    |     |     | 
| all-MiniLM-L6-v2              |    |    |    |     |     | 
| setu4993/LaBSE                |    |    |    |     |     | 

------

**Несколько уточнений по-поводу датасетов:*
1. *Если данные поделены на train/test/val, я оставляю только train.*

2. *Часть данных в не parquet формате я перевёл в parquet для ускорения.*

3. *Я уменьшил размер toxi-text-3M с 2880667 до 37910 семплов. Ограничился 1000 семплов сверху и 100 снизу для каждого языка. Сверху, чтобы я успел провести эксперименты, а снизу, чтобы рассчеты среднего по языкам были более репрезентативны. Для возможности воспроизвести вычисления, файл с обновленным датасетом я добавлю в репозиторий. Кроме того, в ноутбуке **toxic_dataset_preparation.ipynb** находится полный процесс изменения датасета.* 

4. Датасет *sentence_compression* состоит из 180000 семплов, однако большие тексты требуют слишком много времени для вычисления, я ограничился 2000 семплов.


***Во всех метриках я испольхую KNN с параметрами: n_neighbors=11, weights='distance', metric='cosine'. Основательно параметры я не подбирал, кроме числа соседей, в остальном - интуиция.*

In [56]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BertTokenizerFast, BertModel
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer

from src.data_processors import *
from src.model_processors import *
from src.metrics import *

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {DEVICE}')

# Можно изменять порядок и добавлять модели с иным числом весов
def get_models_tokenizers_processors():
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
    model_processor = T5ModelProcessor(model, tokenizer, device=DEVICE)
    yield model_processor
    
    model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')
    tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
    model_processor = E5ModelProcessor(model, tokenizer, device=DEVICE)
    yield model_processor
    
    model = BertModel.from_pretrained("setu4993/LaBSE")
    tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
    model_processor = LabseModelProcessor(model, tokenizer, device=DEVICE)
    yield model_processor
    
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    tokenizer = None
    model_processor = MinilmModelProcessor(model, tokenizer, device=DEVICE)
    yield model_processor

    model = INSTRUCTOR('hkunlp/instructor-base')
    tokenizer = None
    model_processor = InstructorModelProcessor(model, tokenizer, device=DEVICE)
    yield model_processor
    
    model = gensim.downloader.load('glove-twitter-25')
    tokenizer = None
    model_processor = GloveModelProcessor(model, tokenizer, device=DEVICE)
    yield model_processor


def get_dataset_metric():
    dataset_processor = FloresDatasetProcessor()
    metric = calculate_UDM
    yield dataset_processor, metric, None
    
    dataset_processor = ParaphraserDatasetProcessor()
    metric = calculate_PAR
    yield dataset_processor, metric, None
    
    dataset_processor = ToxiDatasetProcessor()
    metric = calculate_TOX
    yield dataset_processor, metric, None
    
    # Ограничиваю 2000, иначе очень долго считать
    dataset_processor = SummDatasetProcessor()
    metric = calculate_SUM
    yield dataset_processor, metric, 2000
    
    dataset_processor = MGPDatasetProcessor()
    metric = calculate_MGP
    yield dataset_processor, metric, None


Using cuda


In [ ]:
import random
from time import time

# Фиксирую случайность 
def seed_all(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

seed_all()
    
start_time = time()

for model_processor in get_models_tokenizers_processors():
    for dataset_processor, metric, k in get_dataset_metric():
        if k is None:
            k = 1e9
        metric_result = metric(model_processor, dataset_processor, k)
        
        eval_time = time() - start_time
        
        with open('logs.txt', 'a') as logs:
            log = f'model:{model_processor.__class__}; metric:{metric.__name__}; result:{metric_result}; eval_time:{eval_time}.\n'
            logs.write(log)

  2%|▊                                      | 752/37910 [00:57<47:37, 13.00it/s]